# Control of a Rocket
According to [1, p. 323], the motion of a rocket can be modeled as follows,

$$\dot{z} =v$$

$$\dot{v} =\frac{u}{m}-g$$

$$\dot{m} =-\alpha u$$

where $m$ is the mass, $v$ is the velocity, $x$ is the vertical displacement, 
$u$ is the control input, $\alpha$ is a constant, and the drag forces have been 
neglected. We are interested in maximizing the range $z(T)$ over a fixed time 
interval $[0, T]$. To regularize the problem we define the following cost to 
be minimized

$$-z(T)+\gamma \int_{0}^{T} u^{2} d t$$

where $\gamma$ is a scalar. Consider the initial conditions $x(0) = 0$, $v(0) 
= 0$, and $m(0) = m_0$ and suppose that $T$ = 4.

This problem can be solved with Pontryagin's maximum principle. Let $x =[\matrix{ 
z & v & m}]^T$ and $\lambda=[\matrix{ \lambda_z & \lambda_v & \lambda_m}]^T$. 
The Hamiltonian is

$$ H(x,u,\lambda) = \lambda^T f(x,u) + g(x,u)$$

where $g(x,u)=u^2$, or equivalently

$$H = \lambda_z v + \lambda_v(\frac{u}{m}-g) +\lambda_m(-\alpha u)+\gamma 
u^2$$

The optimality conditions are, besides the equations of the process,

$$\dot{\lambda}_z = -\frac{\partial H}{\partial z}$$

$$\dot{\lambda}_v = -\frac{\partial H}{\partial v}$$

$$\dot{\lambda}_m = -\frac{\partial H}{\partial m}$$

$$\frac{\partial H}{\partial u} = 0$$

with the terminal condition

$$ \lambda(T) = \frac{\partial g_h(x)}{\partial x}_{|x=x(T)}$$

where $g_h(x)=z$, or equivalently

$$ [\matrix{ \lambda_z(T) & \lambda_v(T) & \lambda_m(T)   		}] = [\matrix{-1 
& 0 & 0  }]$$

Computing the derivatives, we obtain, for $t\in [0,T]$,

$$ \dot{\lambda}_z = 0 \rightarrow \lambda_z(t) =-1$$

$$\dot{\lambda}_v = -\lambda_z \rightarrow \lambda_v(t) =t-T$$

$$\dot{\lambda}_m = -\frac{\partial H}{\partial m} \rightarrow \dot{\lambda}_m 
= \frac{\lambda_v  u}{m^2} \rightarrow \dot{\lambda}_m = \frac{(t-T)  u}{m^2}$$

$$\frac{\partial H}{\partial u} = 0 \rightarrow 2\gamma u + \frac{\lambda_v}{m}-\alpha 
\lambda_m =0\rightarrow u =\frac{1}{2\gamma}(\alpha \lambda_m-\frac{\lambda_v}{m}) 
= \rightarrow u =\frac{1}{2\gamma}(\alpha \lambda_m+\frac{T-t}{m})$$

from which we conclude that

$$  \dot{\lambda}_m = \frac{(t-T)  \frac{1}{2\gamma}(\alpha \lambda_m+\frac{T-t}{m})}{m^2}$$

$$\dot{m} = -\alpha (\frac{1}{2\gamma}(\alpha \lambda_m+\frac{T-t}{m}))$$

with boundary conditions

$$  m(0)=m_0, \ \ \ \ \lambda_m(T)=0.$$

We can solve this system of equations with the shooting method by searching 
over the value of $\lambda_m(0)$ which leads to $\lambda_m(T)=0$. Then, we obtain 
the optimal trajectories

$$ \lambda_m(t), \ \ \ \ m(t), \ \ \ t\in [0,T],$$

and plugging in these expression in the expression for $u$ we obtain an 
optimal trajectories for 

$$u(t), \ \ t \in [0,T],$$

that is,

$$u(t) =\frac{1}{2\gamma}(\alpha \lambda_m(t)+\frac{T-t}{m(t)}).$$

The following function `rocketcontrol` computes $L$ + 1, where $L$ 
= 100, evenly spaced points for the control input $u(t)$.

The inputs to the function are:

* the initial condition for the mass m0 = $m_0$
* the scalar gamma = $\gamma$
* the constant alpha = $\alpha$.

The outputs of this function are:

* the control input u = $\left[\begin{array}{ll}{u(0)} & {u\left(\frac{T}{L}\right)} 
& {u\left(\frac{2 T}{L}\right)}\end{array} \quad \ldots \quad u\left(\frac{(L-1) 
T}{L}\right) \quad u(T)\right]$
* each of the 3 components of the state

u = $\left[\begin{array}{ll}{u(0)} & {u\left(\frac{T}{L}\right)} & {u\left(\frac{2 
T}{L}\right)}\end{array} \quad \ldots \quad u\left(\frac{(L-1) T}{L}\right) 
\quad u(T)\right]$

* maximum range for the horizontal displacement zmax = $z(T)$.

In [ ]:
import numpy as np
import scipy.linalg
from scipy.integrate import solve_ivp
import scipy.optimize
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib ipympl

In [ ]:
def dyn(t,xl,T,gamma,alpha):
    lambdam = xl[0]
    m = xl[1]
    xldot = np.hstack(((t-T)/m**2*1/(2*gamma)*(alpha*lambdam + (T-t)/m),
                        -alpha*(1/(2*gamma)*(alpha*lambdam + (T-t)/m ))))
    return xldot

In [ ]:
def shoot(lambdam0,m0,T,gamma,alpha,tvec): # solve the differential equations for the given lambdao
    sol = solve_ivp(lambda t,xl : dyn(t,xl,T,gamma,alpha) , np.array([0,T]), np.array([lambdam0, m0]), t_eval=tvec)
    return sol.y

In [ ]:
def newton(t,x,u,m,tvec):
    ut = np.interp(t,tvec,u)
    mt = np.interp(t,tvec,m)
    xdot = np.array([x[1],ut/mt])
    return xdot

In [ ]:
def rocketcontrol(m0,gamma,alpha):
    T  = 4
    # in order to speed up the calculations the boundaries of the grid depend on the mass, but in general one can pick sufficiently large boundaries
    
    if m0==1:
        lambdamgrid = np.arange(10,11,0.01)
    elif m0==1.1:
        lambdamgrid = np.arange(31,32,0.01)
    elif m0==1.5:
        lambdamgrid = np.arange(45,48,0.05)
    elif m0==1.6:
        lambdamgrid = np.arange(3,4,0.05)
    elif m0==1.7:
        lambdamgrid = np.arange(0,4,0.01)
    elif m0==2:
        lambdamgrid = np.arange(18,22,0.01)
    

    tau = T/100
    tvec  = np.arange(0,T,tau)
    cost = np.zeros((lambdamgrid.shape[0],1))
    for k1 in range(lambdamgrid.shape[0]):
        lambdam0 = lambdamgrid[k1]
        zlvec = shoot(lambdam0,m0,T,gamma,alpha,tvec)
        cost[k1] = np.linalg.norm(zlvec[0,-1])
    
    mincost = np.min(cost)
    k1min   = np.argmin(cost)
    zlvec   = shoot(lambdamgrid[k1min], m0, T, gamma, alpha, tvec)

    lambdam = zlvec[0,:]
    m = zlvec[1,:]
    u = 1/(2*gamma)*(alpha*lambdam + (T-tvec)/m )

    sol = solve_ivp(lambda t,x : newton(t,x,u,m,tvec) , [0,T], [0,0], method='LSODA',t_eval=tvec)
    x_ = sol.y
    z = x_[0,:]
    v = x_[1,:]
    maxrange = z[-1]

    return u, v, z, m, maxrange

In [ ]:
m0    = 1 # scripts works for values 1, 1.1, 1.5, 1.6, 1.7, 2
gamma = 1
alpha = 1e-6
T  = 4
u, v, z, m, zmax = rocketcontrol(m0,gamma,alpha)
tau = T/100
tvec  = np.arange(0,T,tau)

In [ ]:
f , axes = plt.subplots(1,4)
ax = axes[0]
ax.plot(tvec,z)
ax.set_xlabel('time')
ax.set_ylabel('z')
ax = axes[1]
ax.plot(tvec,v)
ax.set_xlabel('time')
ax.set_ylabel('v')
ax = axes[2]
ax.plot(tvec,u)
ax.set_xlabel('time')
ax.set_ylabel('u')
ax = axes[3]
ax.plot(tvec,m)
ax.set_xlabel('time')
ax.set_ylabel('m');